In [ ]:
import pandas as pd
import ruptures as rpt
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

In [ ]:
#Download fit file 
#convert to csv
#convert to local time

In [ ]:
def process_garmin_data(input_csv, output_csv, time_delta):
    df = pd.read_csv(input_csv)

    #Convert the timestamp to a different timezone
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['local_time'] = df['timestamp'] + pd.Timedelta(hours=time_delta)
    
    #Save the modified DataFrame to a new CSV file
    final_df = df[['timestamp', 'local_time', 'speed']]
    final_df.to_csv(output_csv, index=False)
    
    #Separate the local time into date and time parts
    df2 = pd.read_csv(output_csv)
    df2['local_time'] = pd.to_datetime(df2['local_time'], errors='coerce')
    df2['local_time_part'] = df2['local_time'].dt.time
    df2.to_csv(output_csv, index=False)

    return df2


In [ ]:
#segregate to route segments

In [ ]:
def segment_speed_file(input_csv, output_dir, row_ranges):
    df = pd.read_csv(input_csv)

    #Segment the data based on specified row ranges
    for i, (start, end) in enumerate(row_ranges, start=1):
        segment = df.iloc[start-1:end] 
        segment_output_csv = f"{output_dir}/Segment_{i}.csv"
        
        #Save each segment to a new CSV file
        segment.to_csv(segment_output_csv, index=False)

    return [f"{output_dir}/Segment_{i}.csv" for i in range(1, len(row_ranges)+1)]


In [ ]:
#detect speed change points

In [ ]:
def detect_speed_change_points(input_csv, output_csv, penalty_range, num_penalties):
    df = pd.read_csv(input_csv)
    signal = df["speed"].values
    n = len(signal)

    # calculate AIC
    def calculate_aic(num_params, residual_sum_of_squares):
        likelihood = np.exp(-residual_sum_of_squares / (2 * n))
        return 2 * num_params - 2 * np.log(likelihood)

    # PELT instance
    algo = rpt.Pelt(model="l2").fit(signal)

    #list store results
    penalties = np.linspace(penalty_range[0], penalty_range[1], num_penalties)
    best_aic = np.inf
    best_penalty = None
    best_result = None

    # Find change points for multiple penalty values and calculate AIC
    for pen in penalties:
        result = algo.predict(pen=pen)
        num_params = len(result)  # Number of changepoints
        # Calculate residual sum of squares between the detected segments and the mean value of each segment
        residual_sum_of_squares = 0
        last = 0
        for r in result[:-1]:
            segment = signal[last:r]
            segment_mean = np.mean(segment)
            residual_sum_of_squares += np.sum((segment - segment_mean) ** 2)
            last = r

        aic = calculate_aic(num_params, residual_sum_of_squares)

        if aic < best_aic:
            best_aic = aic
            best_penalty = pen
            best_result = result

    # Retrieve the times corresponding to the detected change points for the best penalty
    change_point_times = [df['local_time_part'][i-1] for i in best_result[:-1]]  # We ignore the last entry as it typically indicates the end of the series

    #Save the CP
    df_change_points = pd.DataFrame(change_point_times, columns=['TimesInSpeedFile'])
    df_change_points.to_csv(output_csv, index=False)

    return best_penalty, best_aic, change_point_times

In [ ]:
#plot speed varaiations as a line graph and speed change points as verticle lines

In [ ]:
def plot_speed_with_vertical_lines(input_csv, specified_times_str):
   
    df = pd.read_csv(input_csv)

    # Step 2: Convert array of change_point_times detected to datetime objects
    specified_times = [datetime.strptime(time, "%H:%M:%S") for time in specified_times_str]

    # convert time string to seconds
    def time_to_seconds(t):
        h, m, s = map(int, t.split(':'))
        return h * 3600 + m * 60 + s

    # Convert 'local_time_part' to seconds
    plot_times_seconds = df['local_time_part'].apply(time_to_seconds)

    # Convert specified times to seconds
    specified_times_seconds = [time_to_seconds(time) for time in specified_times_str]

    # plot
    plt.figure(figsize=(10, 6))
    plt.plot(plot_times_seconds, df['speed'], label='Speed')

    # Draw vertical lines at the specified times (converted to seconds)
    for time in specified_times_seconds:
        plt.axvline(x=time, color='r', linestyle='--', label=f'Change Point {time}')

    # Adjusting x-axis to show time in HH:MM:SS format
    plt.xticks(rotation=45)
    xtick_locs, xtick_labels = plt.xticks()
    xtick_labels = [f'{int(x//3600):02d}:{int((x%3600)//60):02d}:{int(x%60):02d}' for x in xtick_locs]
    plt.xticks(ticks=xtick_locs, labels=xtick_labels)

    plt.xlabel('Time (HH:MM:SS)')
    plt.ylabel('Speed')
    plt.title('Speed vs. Time with Specified Change Points')
    plt.legend()
    plt.show()

In [ ]:
# Find times related to speed change points in cideo file

In [ ]:
def parse_time_to_timedelta(time_str):
    parts = list(map(int, time_str.split(':')))
    if len(parts) == 3:
        return timedelta(hours=parts[0], minutes=parts[1], seconds=parts[2])
    elif len(parts) == 2:
        return timedelta(minutes=parts[0], seconds=parts[1])
    else:
        raise ValueError("Time string format is incorrect. Expected 'HH:MM:SS' or 'MM:SS'.")

def adjust_csv_time(csv_time_str, offset):
    csv_time_delta = parse_time_to_timedelta(csv_time_str)
    adjusted_time = csv_time_delta - offset 
    # Return the adjusted time in 'minute:second' format, ensuring hours are also accounted for if necessary
    total_seconds = adjusted_time.seconds + adjusted_time.days * 24 * 3600
    return f'{total_seconds // 60}:{total_seconds % 60:02d}'

def get_video_time_df(input_csv, output_csv, csv_time_example, video_time_example):
    # Calculate the time offset
    csv_time_delta = parse_time_to_timedelta(csv_time_example)
    video_time_delta = parse_time_to_timedelta(video_time_example)
    time_offset = csv_time_delta - video_time_delta

    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Adjust the times in the CSV file
    df['TimesInVideoFile'] = df['TimesInSpeedFile'].apply(lambda x: adjust_csv_time(x, time_offset))

    # Save the adjusted times to a new CSV file
    df.to_csv(output_csv, index=False)
    return df
